In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_20newsgroups
from tensorflow.keras.layers import Input, Embedding, GRU, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Wybor wszystkich kategorii
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

# Tokenizacja 
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(newsgroups_train.data)
x_train = tokenizer.texts_to_sequences(newsgroups_train.data)
x_test = tokenizer.texts_to_sequences(newsgroups_test.data)

# Ustalenie tej samej dlugosci dla wszystkich danych
max_length = max(len(sequence) for sequence in x_train)
x_train = pad_sequences(x_train, maxlen=max_length)
x_test = pad_sequences(x_test, maxlen=max_length)

# One-hot encode kategorii
y_train = to_categorical(newsgroups_train.target)
y_test = to_categorical(newsgroups_test.target)

# Budowanie modelu
inputs = Input(shape=(max_length,))
x = Embedding(input_dim=10000, output_dim=100, input_length=max_length)(inputs)
x = GRU(units=64, activation='tanh')(x)
x = Dense(units=32, activation='relu')(x)
outputs = Dense(units=20, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

#Kompilacja i trenowanie
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))
